In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import yfinance as yf
from machine_learning_finance import analyze_trades, \
    calc_probabilties_without_lookahead, plot_backtest_analysis, \
    metrics_to_dataframe, create_train_test_windows, make_inverse_env_for
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)
windows = [300, 600, 900, 1500]

symbol = "QQQ"
inverse = "PSQ"
file = f"../backtests/backtest_inverse_{symbol}_365_h0.9_l0.1.csv"
period = 365
pd.options.display.max_rows = None


def plot_win_loss(file, metrics):
    ledger = pd.read_csv(file)
    # Set the style for the plots
    sns.set(style='whitegrid')

    # Create a bar plot for profit and loss stats
    profit_stats, loss_stats = metrics['profit_stats'], metrics['loss_stats']
    profit_loss_df = pd.DataFrame([profit_stats, loss_stats], columns=['min', 'max', 'mean', 'median', 'std'], index=['profit', 'loss'])

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(data=profit_loss_df.transpose(), palette='muted')
    ax.set_title('Profit and Loss Statistics')
    ax.set_ylabel('Value')

    # Show the plot
    plt.show()

def analyze_and_graph_range(file, symbol, start, end, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df, start=start, end=end)
    test_df = calc_probabilties_without_lookahead(test_df, hist_df)
    plot_backtest_analysis(test_df, ledger, inverse=symbol)
    return ledger, test_df
   
def analyze_and_graph(file, symbol, period, df):
    ledger = pd.read_csv(file)
    hist_df = df.head(len(df)-period)
    hist_df = hist_df.tail(period*4)
    df = df.tail(period)
    df = calc_probabilties_without_lookahead(df, hist_df)
    plot_backtest_analysis(df, ledger, inverse=symbol)

    return ledger, df

ticker_obj = yf.download(tickers=symbol)
df = pd.DataFrame(ticker_obj)
#plot_win_loss(file, metrics)

#print(ledger)

#print(metrics_df)
#df.sort_index(ascending=False)

[*********************100%***********************]  1 of 1 completed


In [2]:
ledger, df2 = analyze_and_graph(file, symbol, 365*4, df)    
#ledger, df2 = analyze_and_graph_range(file, symbol, "03-27-2020", "12-17-2021", df)

# Display metrics as text
metrics = analyze_trades(ledger, symbol, period)
metrics["file"] = file
metrics_df = metrics_to_dataframe(metrics) 

merged_df = ledger.merge(df2, left_on='Date', right_index=True, how='left')
merged_df[["Date", "Price", "Product", "Side", "Action","prob_above_trend", "Value"]]

[*********************100%***********************]  1 of 1 completed


,Date,Price,Product,Side,Action,prob_above_trend,Value
0,2021-12-27,10.786800,PSQ,long,enter,0.059026,4938.158399
1,2022-03-07,13.150200,PSQ,long,exit,0.944243,6064.202390
2,2022-03-07,328.108585,QQQ,long,enter,0.944243,5990.567086
3,2022-03-28,368.559104,QQQ,long,exit,0.038965,6760.566057
4,2022-03-28,11.604900,PSQ,long,enter,0.038965,6676.907743
5,2022-05-09,14.018800,PSQ,long,exit,0.907956,8125.778436
6,2022-05-09,300.121494,QQQ,long,enter,0.907956,8025.233492
7,2022-06-02,317.523805,QQQ,long,exit,0.291277,8553.893533
8,2022-06-02,13.119900,PSQ,long,enter,0.291277,8448.073800
9,2022-06-13,14.907600,PSQ,long,exit,0.980566,9670.289249


In [3]:
df2.tail(35)


,Open,High,Low,Close,Adj Close,Volume,prob_above_trend,weighted-volume,trend,long_entry,long_exit,short_entry,short_exit
Date,,,,,,,,,,,,,
2023-04-13,314.829987,319.649994,313.630005,319.170013,319.170013,54613800,0.335705,1.743109e+10,310.784070,NaN,NaN,NaN,NaN
2023-04-14,317.589996,320.359985,315.609985,318.570007,318.570007,56278300,0.378813,1.792858e+10,311.149660,NaN,NaN,NaN,NaN
2023-04-17,318.149994,319.200012,316.040009,318.839996,318.839996,43007000,0.400417,1.371235e+10,311.515251,NaN,NaN,NaN,NaN
2023-04-18,320.989990,321.420013,317.640015,318.859985,318.859985,46746900,0.400788,1.490572e+10,311.880841,NaN,NaN,NaN,NaN
2023-04-19,316.410004,319.790009,316.279999,318.709991,318.709991,38960100,0.391536,1.241697e+10,312.246431,NaN,NaN,NaN,NaN
2023-04-20,315.690002,319.269989,314.970001,316.279999,316.279999,62462100,0.482102,1.975551e+10,312.612022,NaN,NaN,NaN,NaN
2023-04-21,315.910004,317.399994,314.100006,316.609985,316.609985,59002300,0.506281,1.868072e+10,312.977612,NaN,NaN,NaN,NaN
2023-04-24,316.380005,317.739990,313.739990,315.950012,315.950012,44043100,0.548282,1.391542e+10,313.343203,NaN,NaN,NaN,NaN
2023-04-25,314.290009,314.970001,309.890015,309.989990,309.989990,57121600,0.716643,1.770712e+10,313.708793,309.98999,NaN,NaN,309.98999


In [4]:
env = make_inverse_env_for(symbol,
                        inverse,
                        1,
                        period,
                        prob_high=0.9,
                        prob_low=0.1)
env.expert_opinion_df()
env.timeseries.tail(35)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Close,weighted-volume,trend,prob_above_trend,action
Date,,,,,
2023-04-13,0.411982,0.197482,0.334911,0.305738,0
2023-04-14,0.407797,0.205048,0.338526,0.493732,0
2023-04-17,0.409680,0.140922,0.342142,0.558578,0
2023-04-18,0.409820,0.159072,0.345757,0.560793,0
2023-04-19,0.408774,0.121220,0.349372,0.639082,0
2023-04-20,0.391826,0.232835,0.352987,0.698285,0
2023-04-21,0.394127,0.216488,0.356602,0.818828,0
2023-04-24,0.389524,0.144010,0.360218,0.869925,0
2023-04-25,0.347956,0.201680,0.363833,0.962311,1
